In [26]:
import pandas as pd
import pyarrow.parquet as pq
df1=pd.read_csv('/home/shiva/Desktop/datasets/chatting1.csv')
df = pq.read_table(source='/home/shiva/Desktop/datasets/userdata1.parquet').to_pandas()

In [27]:
import re
df['text']=df1['text'].values[:1000]
df['text']=df['text'].apply(lambda x:re.sub('[\d\W]',' ',x))

In [28]:
import pickle 
model=pickle.load(open('Fake_model.pkl','rb'))
# df.to_json('userdata1.json')
df.head()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments,text
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02,Hya
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,,Prasid ager group ta r admin R keu na so ok...
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,,Hmm Kalo Dal Nei Ki levelr stalk korish Ke ...
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,,Ok Bnara bol Sudeshnar sathe ki korechis Ki k...
4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,,Baaltai Dekhabi na to Bujhe gechi Mondarmon...


In [30]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,scan
import time
import pickle,requests

res=requests.get("http://localhost:9200")

model=pickle.load(open('Fake_model.pkl','rb'))

es = Elasticsearch([{'host':"localhost",'port':9200}])
# es = Elasticsearch(connection_class=ThriftConnection)

actions=[
    {
        "_index":"fake_dataset",
        "_type":"doc",
        "_id":id,
        "_source":{
            "text":text,
            "prediction":model.predict([text])
        }
    }
    for id,text in zip(df['id'].values,df['text'].values)
]
st=time.time()
bulk(es,actions)
end=time.time()
print("time taken : ",end-st)

    
# df = pd.concat(map(pd.DataFrame.from_dict, Data), axis=1)['fields'].T
# df.to_csv("File.csv")

time taken :  0.4907681941986084


/home/shiva/python_projects_directory/python_projects_env/lib/python3.5/site-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [32]:
# printing (Scaning) Loaded data
results=scan(es,index="fake_dataset",doc_type="doc",query={"query":{"match_all":{}}})
for item in results:
    print(item['_id'],item['_source'])

/home/shiva/python_projects_directory/python_projects_env/lib/python3.5/site-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


1 {'prediction': [0], 'text': ' Hya'}
2 {'prediction': [0], 'text': ' Prasid ager group ta r admin R keu na   so oke add kora jacche na Delete korish na Ami india gie I ll access it  Yeah okay Time for democracy Time to make everyone admin Tru Tora ki keu Arpan er DP dekhechis   Hmm'}
3 {'prediction': [0], 'text': ' Hmm Kalo Dal Nei Ki levelr stalk korish   Ke   Tui Kothay stalk korlam   Dp ki kore dekhli  Whatsapp e scroll korte giye Accha'}
4 {'prediction': [0], 'text': ' Ok Bnara bol Sudeshnar sathe ki korechis Ki korchi mane   Bondhu hoye Bah Nais   Sotti kotha bol Aare sotti bhai ar kiki chobi tulechis dekha'}
5 {'prediction': [0], 'text': ' Baaltai Dekhabi na to   Bujhe gechi Mondarmoni jaachi   Office grp Jan koto    th n  th Soumya ar Auddy tor group e   December ta tomader sathe Soumya jabe na   biye bari ache'}
6 {'prediction': [0], 'text': ' Auddy jabe Hahahaha Ha sala biyer season eta Charipashe biye Hya bhai Onekdin biyebari te khaini Lokeder pinik newar season Nei kono   

986 {'prediction': [0], 'text': ' aladai Ha Utay star gazing khub bhalo hoy  Utahy ha ami ekhono dekhini eta June best time  video omitted  image omitted Bangladesh theke   Le bara  G'}
987 {'prediction': [0], 'text': ' Test korchi ki korbo  Test korchi ki korbo ha ha ha lol bujhte parchi na lol test test test I just cleared my keyboard history feels so much better   We lost a good man today  image omitted Lutegui Luptegui Loptegui Ki chai  Loptegui Loptegui maach'}
988 {'prediction': [0], 'text': ' Ota lotte maach Arrey Loptgui toh Spain er coach Lopetegui Is he becoming Madrid coach  Yeass                            Biswokaap dekso   image omitted   Haha'}
989 {'prediction': [0], 'text': ' Ha amakeo Monday theke tai korte hobe Egypt bhalo khelche Pleasantly surprised Even without salah Amar baje time Elgin ghum theke uthlam office jabk Spain ta dekhbo office e      e bhalo time hobe  Blue shell  Install zsh'}
990 {'prediction': [0], 'text': ' Never cared for aesthetics   Keno  tui ki

In [43]:
import os
os.